`06_rag`

# RAG(검색-증강 생성) Retrieval-Augmented Generation
- LLM에게 외부 데이터를 컨텍스트로 활용.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
%pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# x = embedding.embed_query('오늘 날씨가 너무 좋아')
# print(len(x))

# VectorStore에 임베딩 후 저장(In memory)
vectorstore = Chroma.from_documents(
    documents, 
    embedding=embedding
)



In [11]:
vectorstore.similarity_search(
    '초보자가 키우기 좋은 애완동물 추천해줘', k=2
)

[Document(id='4c14e0a1-432e-48c5-9561-12acec9e0a0f', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='b72c6c70-4097-4370-8c4c-27e288111902', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

## 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기 (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [13]:
%pip install -q pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()

print('원본 pdf의 장수', len(docs))

원본 pdf의 장수 23


In [22]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500글자당 1 청크 / 50글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

print('분할 후 청크 수', len(split_docs))

분할 후 청크 수 72


In [23]:
# 3. 임베딩, 4. 벡터스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐', k=4
)

[Document(id='253dc9ac-0624-4900-b60f-03325c8c9888', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Docu

## 검색 증강 단계
1. 사용자 질문 (Query)
2. 검색 (Retrieve)
3. LLM
4. 최종 답변

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# Prompt 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성 (retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식 다 가져와')

"삼성전자는 자체 개발 생성 AI '삼성 가우스'를 공개했으며, 온디바이스에서 작동 가능하고 외부 유출 걱정이 없는 안전한 데이터로 학습되었습니다. 이 AI는 언어, 코드, 이미지 모델로 구성되어 있으며, 다양한 제품에 단계적으로 탑재할 계획입니다. 2023년 11월 열린 '삼성 AI 포럼 2023'에서 최초 공개되었으며, 경쟁 디바이스와의 차별화를 위해 발전 중입니다."

In [26]:
%pip install -q pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
# 필요한 라이브러리 설치
# %pip install -q requests
# %pip install -q beautifulsoup4
# %pip install -q faiss-cpu
# %pip install -q langchain-community
# %pip install -q langchain-openai

import os
import requests
from bs4 import BeautifulSoup
import sys

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# 1. 문서 로드 (웹 스크래핑)
urls = [
    "https://ko.wikipedia.org/wiki/%EC%BF%A0%EB%A1%9C%EB%B0%94_%EC%B9%B4%EC%9D%B4%ED%86%A0",
    "https://ko.wikipedia.org/wiki/%EB%AA%85%ED%83%90%EC%A0%95_%EC%BD%94%EB%82%9C%EC%9D%98_%EC%98%81%ED%99%94_%EB%AA%A9%EB%A1%9D"
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

all_text = ""
for url in urls:
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        content = soup.find(id="mw-content-text")
        if content:
            all_text += content.get_text()
    except requests.exceptions.RequestException as e:
        print(f"오류: URL {url}에서 데이터를 가져올 수 없습니다. 오류 메시지: {e}")

# 🚨 개선점 1: all_text가 비어 있을 경우 프로그램 종료
if not all_text.strip():
    print("오류: 웹사이트에서 문서를 찾거나 가져오지 못했습니다. URL을 확인해주세요.")
    sys.exit() # 🚨 프로그램 종료

docs = [Document(page_content=all_text)]

# 2. 문서 분할 (Splitting)
# 🚨 개선점 2: 청크 크기 및 오버랩을 변수로 관리
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
splits = text_splitter.split_documents(docs)

# 3. 임베딩 생성 및 벡터 스토어 구축
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 4. 검색기 생성 (retriever)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 5. 프롬프트 설정
prompt_template = """
너는 주어진 문서 내용을 바탕으로 답변을 제공하는 유용한 도우미야.
만약 맥락에 관련 정보가 없다면, "제공된 문서에서 관련 정보를 찾을 수 없습니다."라고 답변해.
답변은 주어진 문서의 정보를 기반으로 요약해서 작성해.

---
문서 내용: {context}
---
사용자 질문: {question}
"""
custom_prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# 6. LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# 7. RAG 체인 구축
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)

# 8. RAG 실행 및 결과 출력
question = "명탐정 코난에서 괴도 키드가 나온 모든 극장판을 알려줘."

# 디버깅을 위해 retriever가 가져오는 문서 확인
retrieved_docs = retriever.invoke(question)
print("--- retriever가 찾은 문서 내용 ---")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...\n")
print("---------------------------------")

response = chain.invoke(question)
print("RAG 답변:", response)

--- retriever가 찾은 문서 내용 ---
문서 1: 괴도 키드의 코드명 1412. 자세히 보면 영어 KID로 보임.



명탐정 코난에 나오는 쿠로바 도이치의 모습



괴도 키드 그림형 서명 디자인


참고 사항[편집]

↑ 범행으로 인한 피해액은 명탐정 코난 극장판 3기 - 세기말의 마술사 시점에서의 경찰 발표에 따르면 건수는 134건 (미국·프랑스·독일 등 12개국 포함), 도둑맞은 보석류는 152점,...

문서 2: 여러 활동[편집]
 이 부분의 본문은 괴도 키드의 원작 사건 목록입니다.
 매직 카이토의 방영 목록  문서를 참고하십시오.
세상을 시끄럽게 하고 있는 엄청난 도둑이며 요즘답지 않게 예고장을 보내고 예고장대로 보석을 전문적으로 훔친다. 괴도 1412는 18년 전에 파리에서 처음으로 보석을 훔친 것[1]을 시작으로 괴도 활동을 했지만 8년 전부터 활동이 없어졌...

문서 3: 코난을 궁지에 몰아넣은 적도 많지만, 결코 살인을 하지 않고 또 다른 사람의 죽음을 그대로 놓아두지 않으며 좋은 방향으로 생명을 구할 때도 있다. 코난과의 대결에서는 항상 무승부로 끝나는 편으로 키드는 「코난 vs 괴도 키드」란 편에서 코난 ( = 쿠도 신이치 / 남도일)이랑 직접 대면했다. 하지만 이전에 시계탑에서 쿠도 신이치 (남도일)에게 범행[3] 이...

문서 4: ↑ 이름은 번안한 칠흑의 추적자와 달리 이 극장판은 왜색의 문제로 이름도 번안되지 않았다.


외부 링크[편집]
(일본어) 명탐정 코난 극장판 공식 홈페이지
vte아오야마 고쇼의 명탐정 코난만화
단행본 목록
범인 한자와 씨
제로의 일상
경찰학교편
애니메이션
전체 에피소드 목록
1996년~2000년
2001년~2005년
2006년~2010년
2011년~201...

문서 5: 괴도 키드 
제1대괴도키드 : 쿠로바 도이치 (고기영)

성우 - 이케다 슈이치 / 민응식
1대 키드이며, 18년 전부터 8년 전까지 마술사 겸 괴도로 활동하였다. 카이토 (고희도)의 아버지이면서 세계적인 마술사로 8년

In [47]:
# 1. 문서 로드 (PDF)
pdf_path = './data/AI 시대를 견인하는 반도체 산업 전망.pdf'

# 파일이 존재하는지 확인
if not os.path.exists(pdf_path):
    print(f"오류: 지정된 경로에 PDF 파일이 없습니다: {pdf_path}")
    sys.exit()

loader = PyMuPDFLoader(pdf_path)
docs = loader.load()
print('원본 pdf의 총 페이지 수:', len(docs))

# 2. 문서 분할 (Splitting)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)
print('분할 후 청크 수:', len(split_docs))

# 3. 임베딩 생성 및 4. 벡터 스토어 구축
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# 5. 프롬프트 커스터마이징
prompt_template = """
너는 주어진 문서를 요약하는 유용한 도우미야.
답변은 주어진 문서의 정보를 기반으로 2~3문장으로 요약해서 작성해.
문서 내용에 대해 질문하면 안 돼. 오직 요약만 해줘.

---
문서 내용: {context}
---
"""
custom_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context"]
)

# 6. LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# 7. 검색기 최적화
retriever = vectorstore.as_retriever(search_kwargs={'k': 10})

# 8. RAG 체인 구축
chain = (
    {'context': retriever}
    | custom_prompt
    | llm
    | StrOutputParser()
)

# 9. RAG 실행 및 결과 출력
query = 'document summary'

print("-----")


# retriever가 가져오는 문서 확인 (디버깅 용도)
retrieved_docs = retriever.invoke(query)
print("--- 검색된 문서 내용 ---")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1} (내용 시작): {doc.page_content[:150]}...\n")
print("-" * 20)

response = chain.invoke(query)
print("AI 요약 답변:", response)

원본 pdf의 총 페이지 수: 33
분할 후 청크 수: 85
-----
--- 검색된 문서 내용 ---
문서 1 (내용 시작): www.pwc.com/structure for further details. Mentions of Strategy& refer to the global team of practical strategists that is integrated within the PwC n...

문서 2 (내용 시작): Section 4 
고래싸움에서 살아남는 법: 지정학 리스크 대응방안 
반도체 산업의 글로벌 공급망은 지정학적 리스크로 인한 각종 수출 규제 및 국내 수급 강제 
등의 법적 및 정치적 제약에 노출되고 있습니다. 이에 안정적인 공급망 확보를 통한 리스크 
헤징이 각 업체...

문서 3 (내용 시작): Endnotes 
 
1. PwC 2024: Electric Vehicle Sales Review Q2-2024 
2. Omdia analysis and research Q3 2024 
3. AnandTech March 2024: NVIDIA Blackwell Arch...

문서 4 (내용 시작): 제공하는 선제적 전략, 혹은 글로벌 
제품을 유지한 채 필요 시에만 조정하는 
수동적 전략을 선택할 수도 있습니다. 
인재 확보 
성공적인 현지화를 위해 우수한 인재 
확보는 필수적입니다. 업체의 현지화 
수요가 증가하면서, 인재 파이프라인 
구축의 중요성 또한 증가하...

문서 5 (내용 시작): 솔루션을 제공하며, 고객의 성장을 지원하고 있습니다. 
 
본 보고서 작성에 필요한 데이터와 인사이트를 제공해주신 Omdia Semiconductor Research에 깊이 감사드립니다.  
 
 
 
 
 
 
 
 
 
Contacts 
 
Korea  
장유신  ...

문서 6 (내용 시작): Partner, 
PwC US 
amit.dhir@pwc.com 
 
Arup Chatterji  
Partner, 
PwC US 
arup.chat

In [1]:
# Agent + RAG
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_tavily import TavilySearch
from datetime import datetime
# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime('%Y-%m-%d')

llm = ChatOpenAI(model='gpt-4.1-nano')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색합니다.'  # Agent가 언제 이 tool을 쓸지 알게됨
)

text = f"""
너는 웹 검색이 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf 를 검색할 수 있는 어시스턴트야.

- 사용자가 PDF 문서와 관련된 질문(ex. '이 pdf에서', '문서내용', '파일에서)을 하면 반드시 'pdf_search' 도구를 써야해
- 사용자 질문이 팩트체크를 필요로 하고, 최신성이 필요하다 판단되면 web_search를 실행해야해
- 사용자가 일반적인 질문을 하고, 최신성이나 팩트체크가 필요없으면 그냥 답변해
- 뭔가 확실하지 않으면 pdf_search 와 web_search를 모두 실행해서 답변을 생성해

오늘은 {today} 야.
"""


prompt = ChatPromptTemplate.from_messages([
    ('system', text),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory, 
    tools=[search_tool, rag_tool],
    verbose=True)

C:\Users\sec\AppData\Local\Temp\ipykernel_306020\3050205485.py:56: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [3]:
agent_executor.invoke({'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '삼성전자 최신 소식 2025년 9월'}`


{'query': '삼성전자 최신 소식 2025년 9월', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.instagram.com/reel/DN4Xtz5kyv4/', 'title': '삼성전자가 9월 4일 오후 6시 30분에 최신 갤럭시 AI가 탑재된', 'content': "삼성전자는 독일 베를린에서 9월 5일부터 9일까지 개최되는 유럽 최대 가전 전시회 'IFA 2025'에서 고도화된 AI 홈 경험과 혁신적인 AI 가전제품들을 소개", 'score': 0.8446273, 'raw_content': None}, {'url': 'http://www.dtoday.co.kr/news/articleView.html?idxno=727086', 'title': '[빅데이터로본다] 반도체 상장기업 2025년 9월 브랜드평판... 1위 삼성 ...', 'content': '한국기업평판연구소 구창환 소장은 "반도체 상장기업 브랜드평판 2025년 9월 브랜드 빅데이터 분석결과, 삼성전자 ( 대표 전영현 ) 브랜드가 1위를 기록', 'score': 0.70750624, 'raw_content': None}, {'url': 'https://www.businesspost.co.kr/BP?command=article_view&num=408874', 'title': "삼성전자, 'AI 홈 미래 일상을 현실로' 주제로 'IFA 2025' 참가", 'content': "삼성전자는 IFA 2025에서 이 같은 목표 아래 최신 디스플레이 기술이 집약된 '마이크로 RGB TV'와 유럽 고객들을 사로잡을 비스포크 AI 가전 신제품,", 'score': 0.6334335, 'raw_cont

{'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?',
 'chat_history': [HumanMessage(content='삼성전자 소식들 요약해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="최근 삼성전자는 다양한 뉴스와 소식을 전하고 있습니다. 주요 내용은 다음과 같습니다:\n\n1. **신제품 및 기술 혁신**: 삼성전자는 40년 전의 ‘말하는 냉장고’와 같은 AI 가전 혁신을 계속 추진 중이며, 스마트싱스 ‘패밀리 케어’와 같은 가족 연결 기술도 개발하고 있습니다. 또한, 2025년형 TV와 모니터에는 마이크로소프트 '코파일럿'이 탑재되어 있습니다.\n\n2. **신제품 발표 및 행사**: 갤럭시 S25 시리즈, 갤럭시 Z 폴드7, Z 플립7 등 신제품 발표와 함께 다양한 이벤트와 전시회도 마련되고 있습니다. 특히, 갤럭시 게이밍 관련 신기술도 선보이고 있습니다.\n\n3. **반도체 및 기타 기술 동향**: 삼성전자는 2025년 하반기 공채를 실시했고, 반도체 사업에서도 중요한 기술 및 투자가 이루어지고 있습니다. 그러나 최근 미국과 중국 간의 반도체 장비 수출 규제 강화로 인해 일부 주가가 하락하는 등 시장 변화도 관찰되고 있습니다.\n\n4. **글로벌 활동 및 협력**: 해외에서는 스마트 냉장고 및 기타 가전제품, 스타트업 지원, 친환경 활동 등 다양한 활동도 진행되고 있으며, 갤럭시 제품의 다양한 라이브러리와 뉴스도 발표되고 있습니다.\n\n이 외에 더 구체적인 내용이나 특정 분야에 대해 궁금하시면 알려주세요!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="2025년 9월 현재 